In [1]:
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(readr))
suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(beeswarm))
suppressMessages(library(limma))
suppressMessages(library(GenomicFeatures))
suppressMessages(library(Seurat))
suppressMessages(library(data.table))
suppressMessages(library(lmtest))
suppressMessages(library(readxl))

# Deseq

In [31]:
dir <- '/relgamal/'
outdir <- '/relgamal/'

files <- list.files(dir,pattern='_sample_gex_total_counts.txt')
cells <- gsub('_sample_gex_total_counts.txt','', files) 

In [32]:
samples <- c('HPAP-019','HPAP-020','HPAP-021','HPAP-022','HPAP-023','HPAP-024','HPAP-026','HPAP-028','HPAP-029','HPAP-032','HPAP-034','HPAP-035','HPAP-036','HPAP-037','HPAP-038','HPAP-039','HPAP-040','HPAP-042','HPAP-043','HPAP-044','HPAP-045','HPAP-047','HPAP-049','HPAP-050','HPAP-051','HPAP-052','HPAP-053','HPAP-054','HPAP-055','HPAP-056','HPAP-057','HPAP-058','HPAP-059','HPAP-061','HPAP-063','HPAP-064','HPAP-065','HPAP-070','HPAP-071','HPAP-072','HPAP-074','HPAP-075','HPAP-077','HPAP-079','HPAP-080','HPAP-081','HPAP-082','HPAP-083','HPAP-084','HPAP-085','HPAP-087','HPAP-088','HPAP-090','HPAP-091','HPAP-092','HPAP-099','HPAP-100','HPAP-101','HPAP-103','HPAP-104','HPAP-105','HPAP-106','HPAP-107','HPAP-108','HPAP-109')
cond_t1d <- c('HPAP-020','HPAP-021','HPAP-023','HPAP-028','HPAP-032','HPAP-055','HPAP-064','HPAP-071','HPAP-084','HPAP-087')
cond_t2d <- c('HPAP-051','HPAP-057','HPAP-058','HPAP-061','HPAP-065','HPAP-070','HPAP-079','HPAP-081','HPAP-083','HPAP-085','HPAP-088','HPAP-090','HPAP-091','HPAP-100','HPAP-106','HPAP-108', 'HPAP-109')
aab <- c('HPAP-019', 'HPAP-024', 'HPAP-029', 'HPAP-038', 'HPAP-043', 'HPAP-045', 'HPAP-049', 'HPAP-050', 'HPAP-072', 'HPAP-092', 'HPAP-107')
all_disease <- c(cond_t1d, cond_t2d, aab)
nd <- setdiff(samples, all_disease)

In [2]:
npod_matrix <- fread('/Acinar_all_nPOD_sample_gex_total_counts.txt')
hpap_matrix <- fread('/Acinar_sample_gex_total_counts.txt')

count_matrix <- cbind(hpap_matrix,npod_matrix)
count_matrix <- as.data.frame(count_matrix)
count_matrix <- count_matrix[,-which(colnames(count_matrix) %in% cond_t2d)]
count_matrix <- count_matrix[,-grep('multi', colnames(count_matrix))]

In [34]:
count_matrix2 <- count_matrix
count_matrix <- count_matrix[,c(2:ncol(count_matrix))]
rownames(count_matrix) <- count_matrix2$V1

In [35]:
hpap_meta <- fread('/hpap_metadata.tsv')
hpap_meta$condition <- NA
hpap_meta$condition <- as.character(hpap_meta$condition)
hpap_meta[which(hpap_meta$diabetes_status == 'ND'),]$condition <- 'ND'
hpap_meta[which(hpap_meta$diabetes_status == 'Early'),]$condition <- 'T1D_early'
hpap_meta[which(hpap_meta$diabetes_status == 'Late'),]$condition <- 'T1D_late'
hpap_meta[which(hpap_meta$diabetes_status == 'One'),]$condition <- 1
hpap_meta[which(hpap_meta$diabetes_status == 'Multiple'),]$condition <- hpap_meta[which(hpap_meta$diabetes_status == 'Multiple'),]$number_autoantibodies

In [36]:
npod_meta <- fread('/nfs/lab/projects/nPOD/nPOD_metadata.txt')
npod_meta$n_aab <- npod_meta$Condition_subtype

npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'Negative'),]$n_aab <- 0
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == ''),]$n_aab <- 0
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'mIAA+'),]$n_aab <- 1
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'GADA+ IA-2A+ ZnT8A+ mIAA+'),]$n_aab <- 4
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'GADA+IA-2A+'),]$n_aab <- 2
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'GADA+ IA-2A+ ZnT8A+'),]$n_aab <- 3
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'GADA+mIAA+'),]$n_aab <- 2
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'IA-2A+ mIAA*+ ZnT8A+'),]$n_aab <- 3
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'GADA+ZnT8A+'),]$n_aab <- 2
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'GADA+'),]$n_aab <- 1
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'GADA+IA-2A+ZnT8A+'),]$n_aab <- 3
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'GADA+ IA2A+ ZnT8A+'),]$n_aab <- 3
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'IA2A+ mIAA+* ZnT8A+'),]$n_aab <- 3
npod_meta[which(npod_meta$Condition_subtype== 'Aab' & npod_meta$AutoAB_Status == 'GADA+ mIAA+*'),]$n_aab <- 2

npod_meta$chemistry <- '10Xv3'
npod_meta$tissue_source <- 'nPod'
names(npod_meta)[1] <- 'donor_ID'
names(npod_meta)[18] <- 'condition'

In [37]:
hpap_meta$scaled_bmi <- scale(hpap_meta$bmi)
hpap_meta$scaled_age <- scale(hpap_meta$age_years)

npod_meta$scaled_bmi <- scale(npod_meta$BMI)
npod_meta$scaled_age <- scale(npod_meta$Age)

hpap_meta <- hpap_meta[,c('donor_ID','Sex', 'scaled_bmi', 'scaled_age', 'tissue_source', 'chemistry', 'condition')]
npod_meta <- npod_meta[,c('donor_ID','Sex', 'scaled_bmi', 'scaled_age', 'tissue_source', 'chemistry', 'condition')]

hpap_meta$project <- 'HPAP'
npod_meta$project <- 'nPOD'

meta_fin <- rbind(hpap_meta, npod_meta)

In [39]:
nd <- meta_fin[which(meta_fin$condition == 'ND'),]$donor_ID
cond_t1d_early <- meta_fin[which(meta_fin$condition == 'T1D_early'),]$donor_ID
cond_t1d_late <- meta_fin[which(meta_fin$condition == 'T1D_late'),]$donor_ID
cond_t1d <- meta_fin[which(meta_fin$condition2 == 'T1D'),]$donor_ID
cond_aab_one <- meta_fin[which(meta_fin$condition == 1),]$donor_ID
cond_aab_two <- meta_fin[which(meta_fin$condition == 2),]$donor_ID
cond_aab_three <- meta_fin[which(meta_fin$condition == 3),]$donor_ID
cond_aab_multi <- meta_fin[which(meta_fin$condition2 == 'multi'),]$donor_ID

In [3]:
ff <- as.formula ('~ condition + project + scaled_bmi + Sex + scaled_age + chemistry + tissue_source')
rr <- as.formula('~ project + scaled_bmi + Sex + scaled_age + chemistry + tissue_source')

cell <- 'Acinar_all'
raw_counts <- count_matrix
raw_counts <- raw_counts[,(colSums(raw_counts != 0) > 0)]
meta_cell <- subset(meta_fin, donor_ID %in% colnames(raw_counts))
meta_cell <- meta_cell[!duplicated(meta_cell$donor_ID),]

raw_counts <- raw_counts[,which(colnames(raw_counts) %in% meta_fin$donor_ID)]
rownames(raw_counts) <- count_matrix$V1

nd_raw_counts <- raw_counts[which(colnames(raw_counts)%in% nd)]
aab_one_raw_counts <- raw_counts[which(colnames(raw_counts)%in% cond_aab_one)]
aab_two_raw_counts <- raw_counts[which(colnames(raw_counts)%in% cond_aab_two)]
aab_three_raw_counts <- raw_counts[which(colnames(raw_counts)%in% cond_aab_three)]
aab_multi_raw_counts <- raw_counts[which(colnames(raw_counts)%in% cond_aab_multi)]
t1d_raw_counts <- raw_counts[which(colnames(raw_counts)%in% cond_t1d)]
t1d_early_raw_counts <- raw_counts[which(colnames(raw_counts)%in% cond_t1d_early)]
t1d_late_raw_counts <- raw_counts[which(colnames(raw_counts)%in% cond_t1d_late)]

n_nd <- floor(ncol(nd_raw_counts)/2)
n_t1d_late <- floor(ncol(t1d_late_raw_counts)/2)

nd_genes_to_keep0 <- list()
nd_genes_to_keep <- list()
for (a in 1:nrow(nd_raw_counts)){
    test  <- nd_raw_counts[a,] 
    test2  <- as.data.frame(test)
    test2 <- as.numeric(test2)
    test3 <- table(test2)
    test3 <- as.data.frame(test3)
    test3$test2 <- unfactor(test3$test2)
    test3$test2 <- as.numeric(test3$test2)
    test4 <- test3[which(test3$test2 >= 5),]
        if (sum(test4$Freq) >= n_nd){
        nd_genes_to_keep0 <- rownames(test)
        nd_genes_to_keep <- c(nd_genes_to_keep0, nd_genes_to_keep)
        }
    }


t1d_late_genes_to_keep0 <- list()
t1d_late_genes_to_keep <- list()
for (b in 1:nrow(t1d_late_raw_counts)){
    test  <- t1d_late_raw_counts[b,] 
    test2  <- as.data.frame(test)
    test2 <- as.numeric(test2)
    test3 <- table(test2)
    test3 <- as.data.frame(test3)
    test3$test2 <- unfactor(test3$test2)
    test3$test2 <- as.numeric(test3$test2)
    test4 <- test3[which(test3$test2 >= 5),]
    if (sum(test4$Freq) >= n_t1d_late){
        t1d_late_genes_to_keep0 <- rownames(test)
        t1d_late_genes_to_keep <- c(t1d_late_genes_to_keep0, t1d_late_genes_to_keep)
        }
    }

nd_t1d_late <- intersect(nd_genes_to_keep, t1d_late_genes_to_keep)

if ('T1D_late' %in% meta_cell$condition && length(meta_cell$condition[which(meta_cell$condition == 'T1D_late')]) > 1){
    print(cell)
    counts <- raw_counts[which(rownames(raw_counts)%in% nd_aab_three),]

    my_design <- ff
    dds <- DESeqDataSetFromMatrix(round(counts), colData = meta_cell, design = my_design) #colData is where design columns are found
    dds <- estimateSizeFactors(dds)
    dds <- estimateDispersions(dds)
    dds2 <- dds

    ### Pairwise Wald test: conditon vs control  
    dds2 <- nbinomWaldTest(dds2)
    tests <- c('T1D_late')
    for(y in tests){
        res <- results(dds2, contrast=c('condition',y,'ND'))
        res <- as.data.frame(res)
        res <- res[order(res$pvalue),]
        outfile <- paste0( cell, '.deseq.WaldTest.', y , '.tsv')        
    }     
        }

res[which(rownames(res) %in% c('CPA1','PRSS2', 'PNLIPRP1')),]

# DESeq with Continuous Disease Progression

In [11]:
meta_fin[which(meta_fin$condition == 'ND'),]$condition <- 1
meta_fin[which(meta_fin$condition == '2'),]$condition <- 2
meta_fin[which(meta_fin$condition == '2'),]$condition <- 3
meta_fin[which(meta_fin$condition == '3'),]$condition <- 4
meta_fin[which(meta_fin$condition == 'T1D_early'),]$condition <- 5
meta_fin[which(meta_fin$condition == 'T1D_late'),]$condition <- 6
meta_fin$condition <- as.numeric(meta_fin$condition)

cell <- c('Acinar')
raw_counts <- count_matrix
raw_counts <- raw_counts[,(colSums(raw_counts != 0) > 0)]
meta_cell <- subset(meta_fin, donor_ID %in% colnames(raw_counts))
meta_cell <- meta_cell[!duplicated(meta_cell$donor_ID),]
raw_counts <- raw_counts[,which(colnames(raw_counts) %in% meta_fin$donor_ID)]
counts <- raw_counts

if (length(meta_cell$scaled_age) > 1){
    print(cell)

    my_design <- as.formula('~project + scaled_bmi + Sex + scaled_age + chemistry + tissue_source + condition')
    #If no contrast is set in DESeq, default variable it is testing is the last variable in design
    dds  <- DESeqDataSetFromMatrix(round(counts), colData = meta_cell, design = my_design) #colData is where design columns are found
    dds <- DESeq(dds)  #Default's to Wald test, can alternatively select LRT here
    res <- as.data.frame(results(dds))
}

[1] "Acinar"


converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  the design formula contains one or more numeric variables with integer values,
  specifying a model with increasing fold change for higher values.
  did you mean for this to be a factor? if so, first convert
  this variable to a factor using the factor() function

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

2 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest



In [19]:
res2 <- res[which(rownames(res) %in% c('CPA1', 'PNLIPRP1', 'PRSS2')),]
res2$padj <- p.adjust(res2$pvalue, method='BH')
res2

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CPA1,162416.26,-0.21985217,0.06723430,-3.269940,0.001075701,0.003227103
PRSS2,497572.84,-0.15447010,0.09410370,-1.641488,0.100696144,0.151044215
PNLIPRP1,34820.66,-0.06452587,0.05398792,-1.195191,0.232012495,0.232012495


In [4]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /home/relgamal/miniconda3/envs/R-notebooks/lib/libopenblasp-r0.3.24.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: America/Los_Angeles
tzcode source: system (glibc)

attached base packages:
[1] stats4    parallel  stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] readxl_1.4.3                lmtest_0.9-40              
 [3] zoo_1.8-12                  data.table_1.15.4          
 [5] Seurat_5.0.3                SeuratObject_5.0.1         
 